# Inference on Bayesian Network

## Load Data, Create DAG

In [ ]:
library("gRain")
library("Rgraphviz")
library("dagitty")
library("combinat")

options(warn=-1)
# setwd("/Users/apple1/Dropbox/Uni/Nijmegen/WS17_18/BN")
#setwd("D:/RU/Sem1/Bayesian Networks/Project")
setwd("/Users/lisa/Documents/Uni/03.SemesterMaster/BN/bn17")
# setwd("C:/Users/Valentin/Uni/RU/Sem1/Bayesian Networks")
data <- read.table(header=T, sep=",", 'adult_cleaned.csv')


our_network <- dag(~ race:native.country + education:race:sex + workclass:education + hours.per.week:workclass:occupation + occupation:education + marital.status:age + income:age:sex:occupation:hours.per.week)
plot(our_network)
head(data)

data[data=="?"]<-NA
data<-data[complete.cases(data),]

## Find Independencies

In [ ]:
g <- dagitty( "dag{ native.country -> race -> education -> occupation -> hours.per.week -> income; occupation -> income; education -> workclass -> hours.per.week; sex -> education; sex -> income; age -> income; age -> marital.status}" )
independencies <- impliedConditionalIndependencies( g )
print(independencies)

## Test Independencies

In [ ]:
num_test <- 0
num_failed <- 0
all <- length(data[,1])
sig_threshold <- 0.05
# Loop through all found independencies
for(i in 1:length(independencies)){
    # Get variables and conditioning set
    indp <- independencies[i]
    xyz <- unlist(indp) 
    # If conditioning set is empty testing is easy
    if(length(xyz)==2){
        #Chi-square test rmsea correction to test for implied statistical independence
        tst <- chisq.test(data[,xyz[1]], data[,xyz[2]])
        res <- sqrt(max((tst$statistic - tst$parameter)/(( nrow(data) - 1)*tst$parameter),0))
        #If result is larger than 0.05 we reject the null hypothesis of the variables being independent
        if(res > sig_threshold){
            print(paste('test: ', xyz[1], ' and ', xyz[2]))
            print(res)
            num_failed <- num_failed + 1
        }
        num_test <- num_test + 1
    }
    # If conditioning set is not empty testing is more involved
    else{
        failed <- 0
        print(paste(c('test ', xyz[1], ' and ', xyz[2], 'given ', xyz[-(1:2)]), collapse=' '))
        # We have to test for independence given all possible value combinations
        # of the conditioning variables
        strata <- list()
        for(i in 1:length(xyz[-(1:2)])){
                # Get values of variables sorted by frequency
                strata[i] <- as.data.frame(sort(table(data[,xyz[2+i]]),decreasing=T))
        }
        # Get all possible combinations of values
        comb_strata = expand.grid(strata)
        visited <- 0
        idx <- 1
        # Test conditions until 80% of data was tested
        while(visited<0.8*all){
            # Get data given conditioning set
            subdata <- data[interaction(data[,xyz[-(1:2)]]) == interaction(comb_strata[idx,]),]
            x <- subdata[,xyz[1]]
            y <- subdata[,xyz[2]]
            # Only test if enough data (unique values) is available
            if(length(unique(x)) < 2 || length(unique(y)) < 2){
                break
            }
            tst <- chisq.test(x,y)
            res <- sqrt(max((tst$statistic - tst$parameter)/(( nrow(subdata) - 1)*tst$parameter),0))
            if(res > sig_threshold){
                failed <- 1
                print(paste(xyz[-(1:2)], ' = ', comb_strata[idx,]),max.levels=0)
                print(res)
            }
            visited <- visited + nrow(subdata)
            idx <- idx + 1
            num_test <- num_test + 1
        }
        if(failed){
                num_failed <- num_failed + 1
        }
    }
}
print(paste('Number of tests = ', num_test, collapse=''))
print(paste('Number of failed tests = ', num_failed, collapse=''))


## Inference

In [ ]:
## Load deleted rows
del_data <- read.table(header=T, sep=",", 'adult_cleaned_deleted.csv')
new_data <- read.table(header=T, sep=",", 'adult_cleaned_deleted.csv')

In [ ]:
# Create network object
net1 <- compile( grain( our_network, data, smooth=1 ) )
pp <- extractCPT( data, our_network, smooth=1)

### Marginals

In [ ]:
## No evidence income
print('0: <=50k, 1: >50k')
q1 <- querygrain(net1, nodes='income',type = "marginal", evidence = NULL, exclude = TRUE, normalize = TRUE, result = "array", details = 0)
print('Inferred')
print(q1)
print('Frequencies')
t <- table(data['income'])
print(paste(t[[1]] / (t[[1]]+t[[2]]),t[[2]] / (t[[1]]+t[[2]]) , collapse=''))

### Evidences

In [ ]:
## Given White, male
net2 <- setEvidence(net1, c("sex","race"), c("0", "0"))
q2 <- querygrain(net2, nodes='income',type = "marginal")
print(q2)

In [ ]:
## Given Native country: US
net3 <- setEvidence(net1, c("native.country"), c("0"))
q3 <- querygrain(net3, nodes='income',type = "marginal")
print(q3)

In [ ]:
## Female query Education
net4 <- setEvidence(net1, c("sex"), c("1"))
q4 <- querygrain(net4, nodes='education',type = "marginal")
print(q4)

In [ ]:
## Male query Education
net5 <- setEvidence(net1, c("sex"), c("0"))
q5 <- querygrain(net5, nodes='education',type = "marginal")
print(q5)

### Missing values

In [ ]:
head(del_data)

In [ ]:
net6 <- setEvidence(net1, c("age","education","marital.status","race","sex","hours.per.week","native.country","income"), c("4", "2", "0", "1", "0", "5", "1", "1"))
q6 <- querygrain(net6, nodes='workclass',type = "marginal")
q7 <- querygrain(net6, nodes='occupation',type = "marginal")

In [ ]:
print(q6)

In [ ]:
print(q7)

In [ ]:
verbose <- FALSE
var_names <- c('age','workclass','education','marital.status','occupation','race','sex','hours.per.week','native.country','income')
for(i in 1:dim(del_data)[1]){
# for(i in 1:3){
    row <- del_data[i,]
    given_names <- list()
    given_vals <- list()
    missing_names <- list()
    missing_idx <- list()
    g_idx <- 1
    m_idx <- 1
    for(j in 1:length(row)){
        if(is.na(row[[j]])){
            missing_names[[m_idx]] <- var_names[[j]]
            missing_idx[[m_idx]] <- j
            m_idx <- m_idx + 1
        } else {
            given_names[[g_idx]] <- var_names[[j]]
            given_vals[[g_idx]] <- toString(row[[j]])
            g_idx <- g_idx + 1
        }
    }
    net_ev <- setEvidence(net1, unlist(given_names, use.names=FALSE), unlist(given_vals, use.names=FALSE))
    for(j in 1:length(missing_names)){
        q <- querygrain(net_ev, nodes=missing_names[[j]],type = "marginal")
        if(verbose){
            print(q)
            print('Max at')
            print(which.max(unlist(q, use.names=FALSE)))
        }
        new_data[i,missing_idx[[j]]] <- which.max(unlist(q, use.names=FALSE))
    }
    if(verbose){
        print('-----')        
    }
}

In [ ]:
head(new_data)

In [ ]:
head(del_data)

In [ ]:
write.csv(new_data, file = "adult_added_missing_vals.csv")

| | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| **Income** | <=50K | >50K | | | | | | |
| **Sex** | Male | Female | | | | | | |
| **Native Country** | US | Other | | | | | | |
| **Race** | White | Asian-Pac-Islander | Amer-Indian-Eskimo | Other | Black | | | |
| **Occupation** | Exec-managerial | Craft-repair | Other-service | Prof-specialty | Armed-Forces | Sales | | |
|  | Adm-clerical | Handlers-cleaners | Tech-support |  |  | | | |
|  |  | Machine-op-inspct | Priv-house-serv |  |  | | | |
|  |  | Farming-fishing | Protective-serv |  |  | | | |
|  |  | Transport-moving | |  |  | | | |
| **Hours per Week** | 0-15 |16-25 |26-35 | 35-45 | 46-55 |55-99 | | |
| **Education** | Preschool | 9th | Bachelors | Master | Prof-school | Doctorate | | |
|  | 1st-4th | 10th | Some-college |  | |  | | |
|  | 5th-6th | 11th |  |  | |  | | |
|  | 7th-8th | 12th |  |  | |  | | |
|  |  | HS-grad |  |  | |  | | |
|  |  | Assoc-acdm |  |  | |  | | |
|  |  | Assoc-voc |  |  | |  | | |
| **Workclass** | Private | Self-emp-not-inc |Self-emp-inc | Federal-gov | Local-gov |State-gov |Without-pay | Never-worked|
| **Martial Status** | Married-civ-spouse | Divorced | Never-married | Separated | Widowed | Married-spouse-absent | Married-AF-spouse | |
| **Age** | 0-20 | 21-30 | 31-40 | 41-50 | 51-60 | 61-70 | 71-80 | 81-90 |